## Summarize Approval Rate for Interconnection Projects in ERCOT

This notebooks grabs ERCOT interconnection Queue project details and generates a summary table which indicates the approval rate for each technology in a given year.

The methology was adopted from <a href="https://pv-magazine-usa.com/author/william-l-driscoll/">Will Driscol</a>.

In [8]:
# import libararies needed for this notebook
import pandas as pd

### Define Global Variables and Functions For Analysis

In [9]:
BASE_URL = "http://mis.ercot.com/misapp/GetReports.do?reportTypeId=15933&reportTitle=GIS%20Report&showHTMLView=&mimicKey"
"""
URL to the base page where the xlsx sheets are from.

Note: this is unused. It is here as a reference.
"""

LINKS_BY_YEAR_MONTH = {
    '2020-10': "http://mis.ercot.com/misdownload/servlets/mirDownload?mimic_duns=000000000&doclookupId=746380681",
    '2020-09': "http://mis.ercot.com/misdownload/servlets/mirDownload?mimic_duns=000000000&doclookupId=741795478",
    '2020-08': "http://mis.ercot.com/misdownload/servlets/mirDownload?mimic_duns=000000000&doclookupId=736732903",
    '2019-12': "http://mis.ercot.com/misdownload/servlets/mirDownload?mimic_duns=000000000&doclookupId=721696781"
}
"""
Links to ERCOT's interconnection queue data from the most recent months.

These reports are generated monthly, are cumulative and seem to contain
year-to-date data.
"""

SKIPROWS_BY_YEAR_MONTH = {
    '2020-10': 27,
    '2020-09': 27,
    '2020-08': 27,
    '2019-12': 24
}
"""
Number of rows to skip before the header row in the project details tab.
"""

TECH_TYPE_MAP_ERCOT = {
    'BA':'Battery Energy Storage',
    'CC':'Combined-Cycle',
    'CE':'Compressed Air Energy Storage',
    'CP':'Concentrated Solar Power',
    'EN':'Energy Storage',
    'FC':'Fuel Cell',
    'GT':'Gas Combustion Turbine',
    'HY':'Hydroelectric Turbine',
    'IC':'Internal Combustion Engine',
    'OT':'Other',
    'PV':'Photovoltaic Solar',
    'ST':'Steam Turbine non-Combined-Cycle',
    'WT':'Wind Turbine',
}
"""Map of technology type codes to there full names."""


APPROVED_COL = 'fis_approved'
PROPOSED_COL = 'fis_requested'
"""
Column names that indicate proposal and approval date.
"""


def extract_ercot_queue(year_month):
    """
    Read the interconnection xlsx file from a specific month-year.
    
    This method grabs a link from LINKS_BY_YEAR_MONTH to read the
    xlsx file into a `pandas` dataframe. There is only minumal 
    
    All files in LINKS_BY_YEAR_MONTH had the same skiprows and
    sheet_name, but if formatting changes, a little metadata
    mapper would be in order.
    
    Args:
        year_month (string): year-month combo to get. Must be in
            the format: YYYY-MM
            
    Returns:
        pandas.DataFrame: raw table with interconnection project
        details.
    """
    queue_raw_df = (
        pd.read_excel(
            LINKS_BY_YEAR_MONTH[year_month],
            sheet_name="Project Details",
            skiprows=SKIPROWS_BY_YEAR_MONTH[year_month],
            na_values=['Date Not Available']
        )
        .assign(report_date=year_month)
    )
    return queue_raw_df

def simplify_columns(df):
    """
    Simplify column labels for use as snake_case database fields.

    All columns will be re-labeled by:
    * Replacing all non-alphanumeric characters with spaces.
    * Forcing all letters to be lower case.
    * Compacting internal whitespace to a single " ".
    * Stripping leading and trailing whitespace.
    * Replacing all remaining whitespace with underscores.
    
    Note: This function was copied directly from `pudl.helpers`.

    Args:
        df (pandas.DataFrame): The DataFrame to clean.

    Returns:
        pandas.DataFrame: The cleaned DataFrame.

    """
    df.columns = (
        df.columns.
        str.replace('[^0-9a-zA-Z]+', ' ').
        str.strip().
        str.lower().
        str.replace(r'\s+', ' ').
        str.replace(' ', '_')
    )
    return df

def transform_ercot_queue(queue_raw_df):
    """
    Clean the ERCOT Queue project details table.
    
    Note: the cleaning in this function is only that which is required
    for `summarize_approval_rate`. If other analysis is to be done,
    consider adding additional transformation steps.
    
    Args:
        queue_raw_df (pandas.DataFrame): raw table with interconnection
            project details.
    
    Returns:
        pandas.DataFrame: cleaned table with interconnection project
        details.
    
    """
    queue_df = (
        queue_raw_df
        # the header row is five rows wide.... because they didn't
        # learn how to wrap text in an excel cell? idk. most of the column
        # names are in the first header row including all of the columns that
        # we care about for this analysis, so we're just going to drop these
        # mostly empty rows.
        .drop([0,1,2,3], axis='index')
        .pipe(simplify_columns)
        .astype({
            'capacity_mw': float,
            'screening_study_started': 'datetime64[ns]',
            'screening_study_complete': "datetime64[ns]", 
            'fis_requested': "datetime64[ns]",
            'fis_approved':"datetime64[ns]",
        })
        # convert the tech type codes into longer form, more readale strings
        .replace({'technology': TECH_TYPE_MAP_ERCOT})
        # calculate the number of days for approval
        .assign(
            approval_days=lambda x: 
                (x[APPROVED_COL] - x[PROPOSED_COL]).dt.days
        )
    )
    return queue_df

def count_sum_tech_group(df):
    """
    Groupby technology, create a count and sum of capacity of projects.
    
    Args:
        df (pandas.DataFrame): table with the following columns:
            technology, capacity_mw, number_of_projects and approval_days.
    
    Returns:
        pandas.DataFrame: a table grouped by `technology` (indicating the
        technology type of the interconnection projects) including columns
        indicating the sum of the capacity, the number of projects and the
        average of the approval days in each technology types.
    """
    sum_count_df = (
        df
        .assign(number_of_projects=1)
        .groupby(['technology'])
        .agg(
            {'capacity_mw': 'sum',
             'number_of_projects': 'count',
             'approval_days': 'mean'
            })
    )
    return sum_count_df

def summarize_approval_rate(queue_df, year):
    """
    Summarize the approval rate of the queue by technology for a given year.
    
    Grab the dataframes of approved and proposed projects. Run
    `count_sum_tech_group` on both sets of projects. Merge both sets of projects
    together with suffixes. Calculate an average approval rate.
    
    This function assumes that inital a proposed project is 
    
    Args:
        queue_df (pandas.DataFrame): table of interconnection queue project
            details. Result of `transform_ercot_queue()`
        year (int): year for which you'd like to check the interconnection
            approval rate. Format YYYY.
    """
    mask_proposed = queue_df[PROPOSED_COL].dt.year == year
    mask_approved = queue_df[APPROVED_COL].dt.year == year

    proposed_df = queue_df[mask_proposed]
    approved_df = queue_df[mask_approved]

    approval_df = (
        pd.merge(
            count_sum_tech_group(proposed_df).drop(columns='approval_days'),
            count_sum_tech_group(approved_df),
            suffixes=('_proposed','_approved'),
            right_index=True, left_index=True,
            how='outer'
        )
        .assign(
            approval_rate=lambda x:
                x.capacity_mw_approved /x.capacity_mw_proposed)
        .round(2)
    )
    return approval_df

### Run the Code

In [10]:
# which data to grab
data_year_month='2019-12'
# which year to summarize
summary_year=2019

# get the raw table
queue_raw_df = extract_ercot_queue(year_month=data_year_month)
# clean up the table
queue_df = transform_ercot_queue(queue_raw_df=queue_raw_df)
# summarize the table to generate approval rates
approval_rate = summarize_approval_rate(queue_df=queue_df, year=summary_year)

In [11]:
# output the table
approval_rate

,capacity_mw_proposed,number_of_projects_proposed,capacity_mw_approved,number_of_projects_approved,approval_days,approval_rate
technology,,,,,,
Battery Energy Storage,4075.22,43,250.00,2.0,385.00,0.06
Combined-Cycle,182.30,5,363.00,4.0,235.50,1.99
Gas Combustion Turbine,1945.00,7,1893.60,10.0,364.60,0.97
Other,200.00,4,NaN,NaN,NaN,NaN
Photovoltaic Solar,28284.82,109,6002.46,31.0,593.61,0.21
Steam Turbine non-Combined-Cycle,73.00,2,13.00,1.0,41.00,0.18
Wind Turbine,5316.11,25,4898.13,22.0,700.14,0.92


### Export the Output

In [5]:
# export the table to CSV...
# the file will end up in the same folder that this notebook lives in.
# if you are trying to access this via binder, click on the jupyter banner
# to access the files in the `notebooks` directory
approval_rate.to_csv(f'approval_rate_{summary_year}_from_{data_year_month}.csv')